In [9]:
pip install deap

In [10]:
import pandas as pd
import numpy as np
import random
from deap import base, creator, tools

# reproducibilidad opcional
random.seed(1)
np.random.seed(1)

# ----- Parámetros -----
n_partidos = 5
n_curules = 50

# 1) Distribución no uniforme de curules (50)
curules = np.random.multinomial(n_curules, np.random.dirichlet(np.ones(n_partidos)*2))
df_curules = pd.DataFrame({
    "Partido": [f"Partido {i+1}" for i in range(n_partidos)],
    "Curules": curules
})
print("Distribución inicial de curules (poder en el Congreso):")
print(df_curules.to_string(index=False))

# 2) Generar exactamente 50 entidades con peso aleatorio 1-100
n_entidades = 50
entidades = [f"Entidad {i+1}" for i in range(n_entidades)]
pesos = np.random.randint(1, 101, size=n_entidades)  # aleatorio 1..100

df_entidades = pd.DataFrame({
    "Entidad": entidades,
    "Peso": pesos
})

print("\nLista completa de entidades (50) y sus pesos (aleatorios 1-100):")
# Mostrar todas las filas (50)
pd.set_option('display.max_rows', None)
print(df_entidades.to_string(index=False))
pd.reset_option('display.max_rows')

# 3) Preparar AG (objetivo: repartir entidades para ser lo más equitativo posible)
total_curules = df_curules["Curules"].sum()
poder_esperado = [c / total_curules for c in df_curules["Curules"]]  # proporciones objetivo

# Evitar warnings si ya se crearon clases en creator en ejecuciones previas
for name in ("FitnessMin", "Individual"):
    if hasattr(creator, name):
        delattr(creator, name)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # minimizar diferencia
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Representación: lista de largo 50 donde cada valor es 0..4 (partido asignado)
def create_individual():
    return [random.randint(0, n_partidos - 1) for _ in range(n_entidades)]

toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluación: diferencia entre proporción de poder deseada (curules) y proporción lograda (pesos de entidades)
def evaluate(individual):
    poder_partidos = [0] * n_partidos
    for entidad_idx, partido in enumerate(individual):
        poder_partidos[partido] += int(pesos[entidad_idx])  # sumar peso de la entidad al partido
    total_poder_entidades = sum(poder_partidos)
    # si no hay poder (imposible aquí porque pesos>0), proteger división
    if total_poder_entidades == 0:
        proporciones = [0] * n_partidos
    else:
        proporciones = [p / total_poder_entidades for p in poder_partidos]
    # objetivo: minimizar la suma de diferencias absolutas entre proporciones deseadas y obtenidas
    diff = sum(abs(pe - po) for pe, po in zip(poder_esperado, proporciones))
    return (diff,)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=n_partidos-1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# 4) Ejecutar AG
def main():
    pop = toolbox.population(n=200)
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    CXPB, MUTPB = 0.5, 0.2
    NGEN = 300

    for gen in range(NGEN):
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        # cruce
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # mutación
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # reevaluar inválidos
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        pop[:] = offspring

    best = tools.selBest(pop, 1)[0]
    return best

best_solution = main()

# 5) Construir tabla final de asignación
df_result = df_entidades.copy()
df_result["Partido Asignado"] = [f"Partido {p+1}" for p in best_solution]

# Sumar poder (pesos de entidades) por partido y asegurarse de incluir partidos sin asignaciones
partidos_labels = [f"Partido {i+1}" for i in range(n_partidos)]
poder_por_partido = df_result.groupby("Partido Asignado")["Peso"].sum().reindex(partidos_labels, fill_value=0).reset_index()

# Merge con curules
#df_final = df_curules.merge(poder_por_partido, left_on="Partido", right_on="Partido Asignado", how="left")
#df_final = df_final.rename(columns={"Peso": "Poder_Entidades"})
df_final = df_curules.merge(
    poder_por_partido,
    left_on="Partido",
    right_on="Partido Asignado",
    how="left"
).drop(columns=["Partido Asignado"])   # 👈 elimina la columna extra
df_final = df_final.rename(columns={"Peso": "Poder_Entidades"})

df_final["Poder_Entidades"] = df_final["Poder_Entidades"].fillna(0).astype(int)
df_final["Poder_Total (Curules + Peso Entidades)"] = df_final["Curules"] + df_final["Poder_Entidades"]

print("\nAsignación de entidades a partidos (primera 20 filas para referencia):")
print(df_result.head(20).to_string(index=False))

print("\nDistribución final de poder por partido (Curules, Poder_Entidades, Poder_Total):")
print(df_final.to_string(index=False))


Distribución inicial de curules (poder en el Congreso):
  Partido  Curules
Partido 1       23
Partido 2        1
Partido 3        5
Partido 4        2
Partido 5       19

Lista completa de entidades (50) y sus pesos (aleatorios 1-100):
   Entidad  Peso
 Entidad 1    95
 Entidad 2    97
 Entidad 3    87
 Entidad 4    14
 Entidad 5    10
 Entidad 6     8
 Entidad 7    64
 Entidad 8    62
 Entidad 9    23
Entidad 10    58
Entidad 11     2
Entidad 12     1
Entidad 13    61
Entidad 14    82
Entidad 15     9
Entidad 16    89
Entidad 17    14
Entidad 18    48
Entidad 19    73
Entidad 20    31
Entidad 21    72
Entidad 22     4
Entidad 23    71
Entidad 24    22
Entidad 25    50
Entidad 26    58
Entidad 27     4
Entidad 28    69
Entidad 29    25
Entidad 30    44
Entidad 31    77
Entidad 32    27
Entidad 33    53
Entidad 34    81
Entidad 35    42
Entidad 36    83
Entidad 37    16
Entidad 38    65
Entidad 39    69
Entidad 40    26
Entidad 41    99
Entidad 42    88
Entidad 43     8
Entidad 44    27